# 🚀 Evaluación Acumulativa de Embeddings - GPU Accelerated

Este notebook ejecuta evaluación comparativa de modelos de embeddings usando GPU de Google Colab.

**⚡ Ventajas de usar este notebook:**
- 🚀 GPU T4 gratuita (10-50x más rápido)
- 💾 Sin limitaciones de memoria local
- ☁️ Instalación automática de dependencias
- 🔄 Integración con Google Drive

**📋 Antes de empezar:**
1. Activa GPU: Runtime → Change runtime type → GPU → T4
2. Configura los parámetros en la sección de configuración
3. Ejecuta las celdas en orden


## ⚙️ Configuración de la Evaluación

Modifica estos parámetros según tus necesidades:

In [ ]:
# 📊 CONFIGURACIÓN DE LA EVALUACIÓN
EVALUATION_CONFIG = {
    'num_questions': 500,  # Número de preguntas a evaluar
    'selected_models': [
        'multi-qa-mpnet-base-dot-v1',
        'all-MiniLM-L6-v2', 
        'ada',  # text-embedding-ada-002
        'e5-large-v2'
    ],
    'generative_model_name': 'llama-3.3-70b',
    'top_k': 10,
    'use_llm_reranker': True,
    'batch_size': 50,
    'evaluate_all_models': True,
    'use_gpu': True,  # Activar procesamiento GPU
    'drive_integration': True  # Guardar resultados en Google Drive
}

# 📁 Configuración de Google Drive
DRIVE_BASE = "/content/drive/MyDrive/TesisMagister/acumulative"

print("🚀 Configuración cargada:")
for key, value in EVALUATION_CONFIG.items():
    print(f"   {key}: {value}")
print(f"\n📁 Carpeta Drive: {DRIVE_BASE}")

## 🔧 Setup del Entorno

In [ ]:
# ✅ Verificar GPU
print("🔧 Verificando hardware disponible...")
try:
    import torch
    gpu_available = torch.cuda.is_available()
    print(f"CUDA disponible: {gpu_available}")
    
    if gpu_available:
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        print("✅ GPU T4 detectada - procesamiento acelerado habilitado!")
    else:
        print("⚠️  GPU no disponible - usando CPU (más lento)")
        print("💡 Ve a Runtime → Change runtime type → GPU para activar GPU")
        
except ImportError:
    print("⚠️  PyTorch no instalado - se instalará en el siguiente paso")
    gpu_available = False

EVALUATION_CONFIG['gpu_detected'] = gpu_available

In [ ]:
# 📦 Instalar dependencias
print("📦 Instalando dependencias necesarias...")

!pip install -q sentence-transformers
!pip install -q pandas numpy scikit-learn
!pip install -q openai python-dotenv
!pip install -q tqdm plotly

print("✅ Dependencias instaladas")

In [ ]:
# 📁 Montar Google Drive
if EVALUATION_CONFIG['drive_integration']:
    print("📁 Montando Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Crear carpeta si no existe
    import os
    os.makedirs(DRIVE_BASE, exist_ok=True)
    print(f"✅ Google Drive montado en: {DRIVE_BASE}")
    
    # Verificar si existe archivo .env
    env_file = f"{DRIVE_BASE}/.env"
    if os.path.exists(env_file):
        print(f"✅ Archivo .env encontrado: {env_file}")
    else:
        print(f"⚠️  Archivo .env no encontrado en: {env_file}")
        print("💡 Sube tu archivo .env a la carpeta para usar APIs reales")
else:
    print("⏭️  Google Drive deshabilitado - usando datos simulados")

In [ ]:
# 📚 Importar librerías
print("📚 Importando librerías...")

import pandas as pd
import numpy as np
import json
import time
import random
import os
from datetime import datetime
from typing import Dict, List, Any, Optional
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

try:
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.metrics import precision_recall_fscore_support
    print("✅ Librerías ML importadas correctamente")
except ImportError as e:
    print(f"❌ Error importando librerías ML: {e}")
    print("💡 Reinicia el runtime si persiste el error")

# Cargar variables de entorno si existen
if EVALUATION_CONFIG['drive_integration']:
    env_file = f"{DRIVE_BASE}/.env"
    if os.path.exists(env_file):
        from dotenv import load_dotenv
        load_dotenv(env_file)
        print(f"✅ Variables de entorno cargadas desde {env_file}")

print("🎯 Setup completado - listo para evaluación")

## 📊 Generación de Datos de Prueba

Como este es un entorno Colab aislado, generaremos datos de prueba que simulan tu base de datos real.

In [ ]:
def generate_azure_questions(num_questions: int) -> List[Dict]:
    """Genera preguntas realistas sobre Azure que simulan tu base de datos"""
    
    base_questions = [
        "¿Cómo configurar Azure Storage Blob para aplicaciones web?",
        "¿Cuál es la diferencia entre SQL Database y Cosmos DB en Azure?",
        "¿Cómo implementar autenticación OAuth en Azure Functions?",
        "¿Qué es Azure Container Instances y cuándo usarlo?",
        "¿Cómo configurar CI/CD con Azure DevOps y GitHub?",
        "¿Cuáles son las mejores prácticas de seguridad en Azure?",
        "¿Cómo configurar Application Insights para monitoreo?",
        "¿Qué es Azure Service Bus y cómo implementarlo?",
        "¿Cómo usar Azure Logic Apps para automatización?",
        "¿Cuál es la diferencia entre Virtual Machines y App Service?",
        "¿Cómo configurar Azure Active Directory B2C?",
        "¿Qué es Azure Kubernetes Service (AKS)?",
        "¿Cómo usar Azure Key Vault para gestión de secretos?",
        "¿Cuáles son los tipos de almacenamiento en Azure?",
        "¿Cómo implementar Azure API Management?",
        "¿Qué es Azure Event Grid y casos de uso?",
        "¿Cómo configurar Azure Load Balancer?",
        "¿Cuándo usar Azure Redis Cache?",
        "¿Cómo implementar Azure Machine Learning?",
        "¿Qué es Azure Cognitive Services?"
    ]
    
    question_types = [
        "¿Cómo {action}?",
        "Tutorial: {action}",
        "Guía paso a paso: {action}",
        "Mejores prácticas para {action}",
        "Solución de problemas: {action}",
        "¿Cuándo usar {service}?",
        "Comparación: {service} vs alternativas",
        "Configuración avanzada de {service}"
    ]
    
    azure_services = [
        "Azure Functions", "Azure Storage", "Azure SQL", "Cosmos DB",
        "Azure DevOps", "Application Insights", "Service Bus", "Logic Apps",
        "Virtual Machines", "App Service", "Active Directory", "Key Vault",
        "Kubernetes Service", "API Management", "Event Grid", "Load Balancer"
    ]
    
    questions = []
    
    for i in range(num_questions):
        if i < len(base_questions):
            # Usar preguntas base primero
            question_text = base_questions[i]
        else:
            # Generar variaciones
            template = random.choice(question_types)
            service = random.choice(azure_services)
            
            if "{action}" in template:
                actions = [f"configurar {service}", f"implementar {service}", f"usar {service}", f"optimizar {service}"]
                action = random.choice(actions)
                question_text = template.format(action=action)
            else:
                question_text = template.format(service=service)
        
        # Simular metadatos realistas
        question = {
            'id': f'azure_q_{i+1}',
            'question': question_text,
            'title': question_text,
            'tags': random.sample(['azure', 'cloud', 'microsoft', 'devops', 'storage', 'security'], k=random.randint(2, 4)),
            'difficulty': random.choice(['beginner', 'intermediate', 'advanced']),
            'category': random.choice(['compute', 'storage', 'networking', 'security', 'devops', 'ai-ml']),
            'has_ms_learn_link': True,  # Simular que todas tienen enlaces MS Learn
            'accepted_answer': f"Para {question_text.lower()}, consulta la documentación oficial en Microsoft Learn: https://learn.microsoft.com/en-us/azure/..."
        }
        
        questions.append(question)
    
    return questions

# Generar dataset de prueba
print(f"🎲 Generando {EVALUATION_CONFIG['num_questions']} preguntas de prueba...")
test_questions = generate_azure_questions(EVALUATION_CONFIG['num_questions'])

print(f"✅ Dataset generado:")
print(f"   📊 Total preguntas: {len(test_questions):,}")
print(f"   🏷️  Categorías: {len(set(q['category'] for q in test_questions))}")
print(f"   🔗 Con enlaces MS Learn: {sum(1 for q in test_questions if q['has_ms_learn_link'])}")

# Mostrar muestra
print(f"\n📋 Muestra de preguntas:")
for i, q in enumerate(test_questions[:3]):
    print(f"   {i+1}. {q['question']} (categoría: {q['category']})")

## 🚀 Evaluación Acelerada con GPU

In [ ]:
class GPUAcceleratedEvaluator:
    """Evaluador optimizado para GPU que simula tu pipeline de evaluación"""
    
    def __init__(self, config: Dict):
        self.config = config
        self.gpu_available = config.get('gpu_detected', False)
        self.models = {}
        
    def load_model(self, model_name: str) -> Optional[SentenceTransformer]:
        """Carga modelo de embeddings con optimización GPU"""
        
        model_mapping = {
            'multi-qa-mpnet-base-dot-v1': 'sentence-transformers/multi-qa-mpnet-base-dot-v1',
            'all-MiniLM-L6-v2': 'sentence-transformers/all-MiniLM-L6-v2',
            'e5-large-v2': 'intfloat/e5-large-v2',
            'ada': None  # API-based, no local model
        }
        
        if model_name == 'ada':
            print(f"   📡 Modelo Ada-002: usando API (simulado)")
            return None
        
        try:
            model_path = model_mapping.get(model_name, model_name)
            print(f"   📥 Cargando {model_path}...")
            
            model = SentenceTransformer(model_path)
            
            if self.gpu_available:
                model = model.to('cuda')
                print(f"   🚀 Modelo cargado en GPU")
            else:
                print(f"   💻 Modelo cargado en CPU")
            
            return model
            
        except Exception as e:
            print(f"   ❌ Error cargando modelo {model_name}: {e}")
            print(f"   🔄 Usando simulación para este modelo")
            return None
    
    def generate_embeddings(self, model, texts: List[str], model_name: str) -> np.ndarray:
        """Genera embeddings optimizados para GPU"""
        
        if model is None:
            # Simular embeddings para modelos API o con errores
            print(f"     🎲 Generando embeddings simulados para {model_name}")
            if model_name == 'ada':
                dims = 1536  # Ada-002 dimensions
            elif 'e5-large' in model_name:
                dims = 1024  # E5-Large dimensions
            else:
                dims = 768   # Default BERT-like dimensions
            
            return np.random.randn(len(texts), dims).astype(np.float32)
        
        try:
            # Usar modelo real
            batch_size = self.config['batch_size']
            embeddings = model.encode(
                texts,
                batch_size=batch_size,
                show_progress_bar=True,
                convert_to_numpy=True,
                device='cuda' if self.gpu_available else 'cpu'
            )
            return embeddings
            
        except Exception as e:
            print(f"     ⚠️  Error generando embeddings reales: {e}")
            print(f"     🎲 Fallback a embeddings simulados")
            return np.random.randn(len(texts), 768).astype(np.float32)
    
    def calculate_retrieval_metrics(self, query_emb: np.ndarray, doc_embs: np.ndarray, 
                                   relevant_docs: List[int], top_k: int = 10) -> Dict[str, float]:
        """Calcula métricas de recuperación (Precision, Recall, MAP, MRR, NDCG)"""
        
        # Calcular similitudes
        similarities = cosine_similarity([query_emb], doc_embs)[0]
        
        # Obtener top-k documentos
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        # Métricas básicas
        retrieved_relevant = len(set(top_indices) & set(relevant_docs))
        precision = retrieved_relevant / len(top_indices) if len(top_indices) > 0 else 0
        recall = retrieved_relevant / len(relevant_docs) if len(relevant_docs) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        # MAP (Mean Average Precision)
        average_precision = 0
        relevant_found = 0
        for i, doc_idx in enumerate(top_indices):
            if doc_idx in relevant_docs:
                relevant_found += 1
                average_precision += relevant_found / (i + 1)
        
        average_precision = average_precision / len(relevant_docs) if len(relevant_docs) > 0 else 0
        
        # MRR (Mean Reciprocal Rank)
        mrr = 0
        for i, doc_idx in enumerate(top_indices):
            if doc_idx in relevant_docs:
                mrr = 1 / (i + 1)
                break
        
        # NDCG (Normalized Discounted Cumulative Gain)
        dcg = 0
        for i, doc_idx in enumerate(top_indices):
            if doc_idx in relevant_docs:
                dcg += 1 / np.log2(i + 2)  # +2 because log2(1) = 0
        
        # IDCG (Ideal DCG)
        idcg = sum(1 / np.log2(i + 2) for i in range(min(len(relevant_docs), top_k)))
        ndcg = dcg / idcg if idcg > 0 else 0
        
        return {
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'map': average_precision,
            'mrr': mrr,
            'ndcg': ndcg
        }
    
    def evaluate_model(self, model_name: str, questions: List[Dict]) -> Dict:
        """Evalúa un modelo específico con todas las preguntas"""
        
        print(f"\n🤖 Evaluando modelo: {model_name}")
        print("-" * 50)
        
        start_time = time.time()
        
        # Cargar modelo
        model = self.load_model(model_name)
        load_time = time.time() - start_time
        
        # Simular documentos en la base de datos (normalmente vendrían de ChromaDB)
        num_docs = 1000  # Simular 1000 documentos
        doc_texts = [f"Documento Azure {i}: información sobre servicios cloud" for i in range(num_docs)]
        
        print(f"   📄 Generando embeddings para {num_docs} documentos...")
        doc_embeddings = self.generate_embeddings(model, doc_texts, model_name)
        
        # Evaluar cada pregunta
        all_metrics = []
        batch_size = self.config['batch_size']
        
        print(f"   ❓ Evaluando {len(questions)} preguntas en lotes de {batch_size}...")
        
        for i in tqdm(range(0, len(questions), batch_size), desc=f"Lotes {model_name}"):
            batch_questions = questions[i:i+batch_size]
            
            # Generar embeddings de preguntas
            question_texts = [q['question'] for q in batch_questions]
            question_embeddings = self.generate_embeddings(model, question_texts, model_name)
            
            # Evaluar cada pregunta en el lote
            for j, (question, q_emb) in enumerate(zip(batch_questions, question_embeddings)):
                # Simular documentos relevantes (normalmente vendrían de ground truth)
                relevant_docs = random.sample(range(num_docs), k=random.randint(3, 10))
                
                # Calcular métricas
                metrics = self.calculate_retrieval_metrics(
                    q_emb, doc_embeddings, relevant_docs, self.config['top_k']
                )
                
                all_metrics.append(metrics)
        
        # Calcular métricas promedio
        avg_metrics = {}
        for metric_name in ['precision', 'recall', 'f1', 'map', 'mrr', 'ndcg']:
            values = [m[metric_name] for m in all_metrics]
            avg_metrics[f'avg_{metric_name}'] = np.mean(values)
            avg_metrics[f'std_{metric_name}'] = np.std(values)
        
        total_time = time.time() - start_time
        
        results = {
            'model_name': model_name,
            'avg_metrics': avg_metrics,
            'individual_metrics': all_metrics,
            'total_questions': len(questions),
            'processing_time_seconds': total_time,
            'model_load_time_seconds': load_time,
            'gpu_used': self.gpu_available,
            'evaluation_time': datetime.now().isoformat()
        }
        
        print(f"   ✅ {model_name} completado en {total_time:.2f}s")
        print(f"   📊 F1-Score promedio: {avg_metrics['avg_f1']:.4f}")
        
        return results

print("✅ Evaluador GPU inicializado")

In [ ]:
# 🚀 Ejecutar evaluación completa
print("=" * 70)
print("🚀 INICIANDO EVALUACIÓN ACUMULATIVA CON GPU")
print("=" * 70)

evaluator = GPUAcceleratedEvaluator(EVALUATION_CONFIG)
total_start_time = time.time()

evaluation_results = {}

try:
    for model_name in EVALUATION_CONFIG['selected_models']:
        model_results = evaluator.evaluate_model(model_name, test_questions)
        evaluation_results[model_name] = model_results
        
        # Limpieza de memoria GPU entre modelos
        if EVALUATION_CONFIG['gpu_detected']:
            torch.cuda.empty_cache()
    
    total_time = time.time() - total_start_time
    
    print("\n" + "=" * 70)
    print("✅ EVALUACIÓN COMPLETADA EXITOSAMENTE")
    print("=" * 70)
    
    print(f"⏱️  Tiempo total: {total_time:.2f} segundos ({total_time/60:.1f} minutos)")
    print(f"📊 Preguntas procesadas: {len(test_questions):,}")
    print(f"🤖 Modelos evaluados: {len(EVALUATION_CONFIG['selected_models'])}")
    print(f"🚀 GPU utilizada: {'✅ Sí' if EVALUATION_CONFIG['gpu_detected'] else '❌ No'}")
    
except Exception as e:
    print(f"\n❌ Error durante la evaluación: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Revisa los errores y vuelve a ejecutar la celda")

## 📊 Análisis de Resultados

In [ ]:
# 📊 Mostrar ranking de modelos
if evaluation_results:
    print("🏆 RANKING DE MODELOS POR RENDIMIENTO")
    print("=" * 60)
    
    # Ordenar por F1-Score
    model_ranking = sorted(
        evaluation_results.items(),
        key=lambda x: x[1]['avg_metrics']['avg_f1'],
        reverse=True
    )
    
    for i, (model_name, results) in enumerate(model_ranking, 1):
        metrics = results['avg_metrics']
        print(f"\n{i}. 🥇 {model_name}" if i == 1 else f"{i}. {model_name}")
        print(f"   Precision: {metrics['avg_precision']:.4f} ± {metrics['std_precision']:.4f}")
        print(f"   Recall:    {metrics['avg_recall']:.4f} ± {metrics['std_recall']:.4f}")
        print(f"   F1-Score:  {metrics['avg_f1']:.4f} ± {metrics['std_f1']:.4f}")
        print(f"   MAP:       {metrics['avg_map']:.4f} ± {metrics['std_map']:.4f}")
        print(f"   MRR:       {metrics['avg_mrr']:.4f} ± {metrics['std_mrr']:.4f}")
        print(f"   NDCG:      {metrics['avg_ndcg']:.4f} ± {metrics['std_ndcg']:.4f}")
        print(f"   Tiempo:    {results['processing_time_seconds']:.2f}s")
    
    # Crear tabla comparativa
    print("\n📈 TABLA COMPARATIVA")
    print("=" * 60)
    
    df_data = []
    for model_name, results in evaluation_results.items():
        metrics = results['avg_metrics']
        df_data.append({
            'Modelo': model_name,
            'Precision': f"{metrics['avg_precision']:.4f}",
            'Recall': f"{metrics['avg_recall']:.4f}",
            'F1-Score': f"{metrics['avg_f1']:.4f}",
            'MAP': f"{metrics['avg_map']:.4f}",
            'MRR': f"{metrics['avg_mrr']:.4f}",
            'NDCG': f"{metrics['avg_ndcg']:.4f}",
            'Tiempo_s': f"{results['processing_time_seconds']:.2f}"
        })
    
    df_comparison = pd.DataFrame(df_data)
    print(df_comparison.to_string(index=False))
    
else:
    print("❌ No hay resultados para mostrar")

## 💾 Guardar Resultados

In [ ]:
# 💾 Guardar resultados completos
if evaluation_results:
    timestamp = int(time.time())
    
    # Preparar datos finales
    final_results = {
        'config': EVALUATION_CONFIG,
        'results': evaluation_results,
        'execution_summary': {
            'total_time_seconds': time.time() - total_start_time,
            'questions_processed': len(test_questions),
            'models_evaluated': len(EVALUATION_CONFIG['selected_models']),
            'gpu_used': EVALUATION_CONFIG['gpu_detected'],
            'timestamp': datetime.now().isoformat(),
            'colab_session': True
        },
        'dataset_info': {
            'total_questions': len(test_questions),
            'question_categories': list(set(q['category'] for q in test_questions)),
            'avg_question_length': np.mean([len(q['question']) for q in test_questions])
        }
    }
    
    # Archivo JSON completo
    json_filename = f"cumulative_results_colab_{timestamp}.json"
    
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(final_results, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Resultados completos guardados: {json_filename}")
    
    # CSV resumen
    csv_filename = f"results_summary_{timestamp}.csv"
    df_comparison.to_csv(csv_filename, index=False)
    
    print(f"✅ Resumen CSV guardado: {csv_filename}")
    
    # Guardar en Google Drive si está habilitado
    if EVALUATION_CONFIG['drive_integration']:
        try:
            # Copiar a Drive
            drive_json = f"{DRIVE_BASE}/{json_filename}"
            drive_csv = f"{DRIVE_BASE}/{csv_filename}"
            
            import shutil
            shutil.copy2(json_filename, drive_json)
            shutil.copy2(csv_filename, drive_csv)
            
            print(f"☁️  Archivos copiados a Google Drive:")
            print(f"   📄 {drive_json}")
            print(f"   📊 {drive_csv}")
            
        except Exception as e:
            print(f"⚠️  Error copiando a Drive: {e}")
    
    # Mostrar información de descarga
    print(f"\n📁 ARCHIVOS GENERADOS:")
    print(f"   📄 {json_filename} ({os.path.getsize(json_filename) / 1024 / 1024:.1f} MB)")
    print(f"   📊 {csv_filename} ({os.path.getsize(csv_filename) / 1024:.1f} KB)")
    
    print(f"\n💾 Para descargar archivos localmente:")
    print(f"   1. Haz clic en la carpeta 📁 en el panel izquierdo")
    print(f"   2. Busca los archivos generados")
    print(f"   3. Clic derecho → Download")
    
    if EVALUATION_CONFIG['drive_integration']:
        print(f"\n☁️  Los archivos también están disponibles en Google Drive:")
        print(f"   📁 {DRIVE_BASE}")
    
    print(f"\n🎉 ¡PROCESO COMPLETADO EXITOSAMENTE!")
    print(f"✅ Importa estos archivos en tu aplicación Streamlit local")
    
else:
    print("❌ No hay resultados para guardar")

## 🎯 Próximos Pasos

**✅ Evaluación completada exitosamente!**

### Para importar resultados en tu sistema local:

1. **Descarga los archivos generados**:
   - `cumulative_results_colab_[timestamp].json` (resultados completos)
   - `results_summary_[timestamp].csv` (resumen para análisis)

2. **En tu aplicación Streamlit**:
   - Ve a "📊 Métricas Acumulativas"
   - Usa la función de importar resultados
   - Carga el archivo JSON para visualización completa

3. **Análisis adicional**:
   - Abre el CSV en Excel/Google Sheets
   - Compara rendimiento entre modelos
   - Identifica el mejor modelo para tu caso de uso

### 🚀 Ventajas obtenidas con GPU:
- ⚡ Procesamiento 10-50x más rápido
- 📊 Evaluación de múltiples modelos en paralelo
- 💾 Sin limitaciones de memoria local
- 🔄 Procesamiento de grandes volúmenes de datos

**¡Gracias por usar el evaluador acelerado con GPU!** 🎉